In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.layers import *
import tensorlayer as tl
from Preprocessing import *
import time
from cv2 import cv2 as cv
from model import *

In [2]:
task = "all"

In [3]:
save_dir = "checkpoint"
tl.files.exists_or_mkdir(save_dir)
tl.files.exists_or_mkdir("samples/{}".format(task))

[TL] [!] checkpoint exists ...
[TL] [!] samples/all exists ...


True

In [4]:
import dataset

28
13
28
(1, 240, 240, 153)
(1, 240, 240, 153)
(1, 240, 240, 153)
(1, 240, 240, 153)
{'DWI': {'mean': 21.078682, 'std': 44.95277}, 'Flair': {'mean': 24.302284, 'std': 50.49608}, 'T1': {'mean': 31.60154, 'std': 63.932735}, 'T2': {'mean': 44.153976, 'std': 92.98761}}
 HGG Validation
finished 12
finished 13
 HGG Train
finished 1
finished 2
finished 3
finished 4
finished 5
finished 6
finished 7
finished 8
finished 9
finished 10
finished 11
finished 12
(1836, 240, 240, 4)
(1836, 240, 240)


In [5]:
XTrain = dataset.X_train_input
yTrain = dataset.X_train_target[:, :, :, np.newaxis]
XTest = dataset.X_dev_input
yTest = dataset.X_dev_target[:, :, :, np.newaxis]

In [6]:
yTrain = (yTrain > 0).astype(int)
yTest = (yTest > 0).astype(int)

In [7]:
batch_size = 10
lr = 0.0001
n_epoch = 120
evalEpoch = 100
dropout_value = 0.05
print_freq_step = 100

In [11]:
tf.reset_default_graph()
with tf.device("/cpu"):
    sess = tf.Session(config = tf.ConfigProto(allow_soft_placement = True))
    with tf.device("/gpu"):
        
        XInput = tf.placeholder(dtype = tf.float32, shape = [batch_size, 240, 240, 4], name = "input_image")
        yInput = tf.placeholder(dtype = tf.float32, shape = [batch_size, 240, 240, 1], name = "ground_truth")
        dropout_ph = tf.placeholder_with_default(tf.cast(dropout_value, dtype = tf.float32), shape = [], name = "dropout_probability")
        
        yOutput = UNetWithAttention(XInput, reuse = False, n_out = 1, dropout_ph = dropout_ph)
        
        yOutputTest = UNetWithAttention(XInput, reuse = True, n_out = 1, dropout_ph = dropout_ph)
        
        # defining loss
        loss = 1 - tl.cost.dice_coe(yOutput, yInput, axis = [0, 1, 2, 3])
        testLoss = 1 - tl.cost.dice_coe(yOutputTest, yInput, axis = [0, 1, 2, 3])
    
    
    trainVar = tl.layers.get_variables_with_name('u_net_attention', True, True)
    with tf.device('/gpu'):
        with tf.variable_scope('learning_rate'):
            lrVar = tf.Variable(lr, trainable = False)
        
        optimizer = tf.train.AdamOptimizer(lrVar).minimize(loss, var_list = trainVar)
    
    sess.run(tf.global_variables_initializer())
    # load existing model if exist
    saver = tf.train.Saver(var_list = trainVar)

 * Input: size of image: 240 240 4
 * Input: size of image: 240 240 4
[TL]   [*] geting variables with u_net_attention
[TL]   got   0: u_net_attention/conv1_l1_1/kernel:0   (3, 3, 4, 64)
[TL]   got   1: u_net_attention/conv1_l1_1/bias:0   (64,)
[TL]   got   2: u_net_attention/batch_normalization/gamma:0   (64,)
[TL]   got   3: u_net_attention/batch_normalization/beta:0   (64,)
[TL]   got   4: u_net_attention/convl1_2/kernel:0   (3, 3, 4, 64)
[TL]   got   5: u_net_attention/convl1_2/bias:0   (64,)
[TL]   got   6: u_net_attention/batch_normalization_1/gamma:0   (64,)
[TL]   got   7: u_net_attention/batch_normalization_1/beta:0   (64,)
[TL]   got   8: u_net_attention/conv1_l2_1/kernel:0   (3, 3, 64, 128)
[TL]   got   9: u_net_attention/conv1_l2_1/bias:0   (128,)
[TL]   got  10: u_net_attention/batch_normalization_2/gamma:0   (128,)
[TL]   got  11: u_net_attention/batch_normalization_2/beta:0   (128,)
[TL]   got  12: u_net_attention/convl2_2/kernel:0   (3, 3, 64, 128)
[TL]   got  13: u_net

In [12]:
# Train
if os.path.exists("./checkpoint/u_net_attention_{}.ckpt".format(task)) == True:
    saver.restore(sess, "./checkpoint/u_net_attention_{}.ckpt".format(task))
    print("[+] Presaved model restored")
else:
    print("[+] Preloaded model not available")
for epoch in range(0, n_epoch + 1):
    epochTime = time.time()
    total_loss, n_batch = 0, 0
    for batch in tl.iterate.minibatches(inputs=XTrain, targets=yTrain, batch_size=batch_size, shuffle=True):
        images, labels = batch
        step_time = time.time()
        ## data augumentation for a batch of Flair, T1, T1c, T2 images
        # and label maps synchronously.
        data = tl.prepro.threading_data([_ for _ in zip(images[:,:,:,0, np.newaxis],
                images[:,:,:,1, np.newaxis], images[:,:,:,2, np.newaxis],
                images[:,:,:,3, np.newaxis], labels)],
                fn = DistortImages) # (10, 5, 240, 240, 1)
        b_images = data[:,0:4,:,:,:]  # (10, 4, 240, 240, 1)
        b_labels = data[:,4,:,:,:]
        b_images = b_images.transpose((0,2,3,1,4))
        b_images.shape = (batch_size, 240, 240, 4)

        ## update network
        _, _loss, out = sess.run([optimizer, loss, yOutput], feed_dict = {XInput:b_images, yInput:b_labels})
        total_loss += _loss
        n_batch += 1
        print("Epoch %d step %d 1 - dice: %f time taken: %fsecs          " %(epoch, n_batch, _loss, time.time() - step_time), end = "\r")

        if np.isnan(_loss):
            exit(" ** NaN loss found during training, stop training")

        if np.isnan(out).any():
            exit(" ** NaN found in output images during training, stop training")
                
    print("\n ** Epoch [%d/%d] trained ==> loss: %f -- time taken: %fsec" %(epoch, n_epoch, total_loss/n_batch, time.time() - epochTime))
    
    ## save a predition of training set
    for i in range(batch_size):
        if np.max(b_images[i]) > 0:
            VisualizeImageWithPrediction(b_images[i], b_labels[i], out[i], "samples/{}/train_{}.png".format(task, epoch))
            break
        elif i == batch_size-1:
            VisualizeImageWithPrediction(b_images[i], b_labels[i], out[i], "samples/{}/train_{}.png".format(task, epoch))
    
    # EVALUATE
    total_loss, n_batch = 0, 0
    for batch in tl.iterate.minibatches(inputs=XTest, targets=yTest, batch_size=batch_size, shuffle=True):
        b_images, b_labels = batch
        _loss, out = sess.run([testLoss, yOutputTest],{XInput: b_images, yInput: b_labels})
        total_loss += _loss
        n_batch += 1

    print(" **"+" "*17+"test -- loss: %f" %(total_loss/n_batch))
    print(" task: {}".format(task))
    ## save a predition of test set
    for i in range(batch_size):
        if np.max(b_images[i]) > 0:
            VisualizeImageWithPrediction(b_images[i], b_labels[i], out[i], "samples/{}/test_{}.png".format(task, epoch))
            break
        elif i == batch_size-1:
            VisualizeImageWithPrediction(b_images[i], b_labels[i], out[i], "samples/{}/test_{}.png".format(task, epoch))
    
    savePath = saver.save(sess, "./checkpoint/u_net_attention_{}.ckpt".format(task))
    print("[+] Checkpoint saved in ", savePath)

[+] Preloaded model not available
Epoch 0 step 183 1 - dice: 0.395187 time taken: 1.285937secs          
 ** Epoch [0/120] trained ==> loss: 0.443281 -- time taken: 240.276938sec
 **                 test -- loss: 0.772260
 task: all
[+] Checkpoint saved in  ./checkpoint/u_net_attention_all.ckpt


KeyboardInterrupt: 

In [13]:
sess.close()

In [ ]:
# %load metrics.py
# Evaluation Metrics - TP, TN, FP, FN, Precision, Recall, Accuracy, F_Score
import numpy as np
import pandas as pd

def metric(prediction, ground_truth, metrics):
    
    #True Positive (TP): we predict a label of 1, true label is 1.
    TP_i = np.sum(np.logical_and(prediction == 1, ground_truth == 1))

    # True Negative (TN): we predict a label of 0 (negative), and the true label is 0.
    TN_i = np.sum(np.logical_and(prediction == 0, ground_truth == 0))

    # False Positive (FP): we predict a label of 1 (positive), but the true label is 0.
    FP_i = np.sum(np.logical_and(prediction == 1, ground_truth == 0))

    # False Negative (FN): we predict a label of 0 (negative), but the true label is 1.
    FN_i = np.sum(np.logical_and(prediction == 0, ground_truth == 1))

    P_i = TP_i/(TP_i+FP_i) #Precision for ith test-reference sample
    R_i = TP_i/(TP_i+FN_i) #Recall for ith test-reference sample
    F_score_i = 2*P_i*R_i/(P_i+R_i) #F_score for ith test-reference sample
    Accuracy_i = (TP_i+TN_i)/(TP_i+TN_i+FP_i+FN_i) #Accuracy for ith test-reference sample
    

    #Confusion matrix for ith example
    metrics["Confusion"][0].append(TP_i)
    metrics["Confusion"][1].append(FP_i)
    metrics["Confusion"][2].append(FN_i)
    metrics["Confusion"][3].append(FP_i)
    
    #Appending metrics of ith sample
    metrics["Precision"].append(P_i)
    metrics["Recall"].append(R_i)
    metrics["F_score"].append(F_score_i)
    metrics["Accuracy"].append(Accuracy_i)

def avg_metrics(metrics):
    metrics_avg = {}
    metrics_avg["Accuracy_avg"] = np.sum(metrics["Accuracy"])/np.size(metrics["Accuracy"])
    metrics_avg["Precision_avg"] = np.sum(metrics["Precision"])/np.size(metrics["Precision"])
    metrics_avg["F_score_avg"] = np.sum(metrics["F_score"])/np.size(metrics["F_score"])
    metrics_avg["Recall_avg"] = np.sum(metrics["Recall"])/np.size(metrics["Recall"])
    metrics_avg["TP"] = np.sum(metrics["Confusion"][0])/np.size(metrics["Confusion"][0])
    metrics_avg["FP"] = np.sum(metrics["Confusion"][1])/np.size(metrics["Confusion"][1])
    metrics_avg["FN"] = np.sum(metrics["Confusion"][2])/np.size(metrics["Confusion"][2])
    metrics_avg["TN"] = np.sum(metrics["Confusion"][3])/np.size(metrics["Confusion"][3])
    return metrics_avg

def exportcsv(metric_dict):
    df = pd.DataFrame(data=[metric_dict])
    df.to_csv("./metrics.csv", sep=',',index=False)

In [ ]:
#Sample Pairs of Predictions & Ground_Truth Images of size 240*2
im_1=np.random.randint(2, size=(240, 240, 1))
im_2=np.random.randint(2, size=(240, 240, 1))
im_3=np.random.randint(2, size=(240, 240, 1))
im_4=np.random.randint(2, size=(240, 240, 1))
im_5=np.random.randint(2, size=(240, 240, 1))
im_6=np.random.randint(2, size=(240, 240, 1))
m = {'Precision':[], 'Recall':[], 'Accuracy':[], 'F_score':[], 'Confusion':[[],[],[],[]]}
metric(im_1, im_2, m)
metric(im_3, im_4, m)
metric(im_5, im_6, m)
m_avg = avg_metrics(m)

In [ ]:
m

In [ ]:
m_avg

In [7]:
df = pd.DataFrame(data=[m_avg])
df

,Accuracy_avg,FN,FP,F_score_avg,Precision_avg,Recall_avg,TN,TP
0,0.499907,14455.333333,14350.0,0.500081,0.501,0.499167,14350.0,14407.333333
